## Summing all the prime numbers below a given number

In [ ]:
import sys
import numba

# Default values (for Jupyter)
number = 2_500_000
ncores = 1

try:
    number = int(sys.argv[1])
except:
    pass

try:
    ncores = int(sys.argv[2])
except:
    pass

numba.set_num_threads(ncores)

print(f"Number: {number}")
print(f"Cores: {ncores}")

In [ ]:
import time

# Código original: Python secuencial

def if_prime(x):
    if x <= 1:
        return 0
    elif x <= 3:
        return x
    elif x % 2 == 0 or x % 3 == 0:
        return 0
    i = 5
    while i**2 <= x:
        if x % i == 0 or x % (i + 2) == 0:
            return 0
        i += 6
    return x

def sum_primes(x):
    result = 0
    for i in range(x):
        result += if_prime(i)
    return result

if ncores == 1:
    print("\n========== Python sequential ==========")
    suma = 0
    N = 3

    start = time.time()
    for i in range(N):
        suma = sum(map(if_prime, range(number)))
    stop = time.time()

    tiempo = (stop - start) / N

    print("The prime sum below ", number, "is ", suma,
          " and the time taken is", tiempo)
else:
    print("Skipping Python sequential version (ncores > 1)")

In [ ]:
from numba import njit

# Optimización 1: Numba secuencial

@njit
def if_prime_numba(x):
    if x <= 1:
        return 0
    elif x <= 3:
        return x
    elif x % 2 == 0 or x % 3 == 0:
        return 0
    i = 5
    while i * i <= x:
        if x % i == 0 or x % (i + 2) == 0:
            return 0
        i += 6
    return x

@njit
def sum_primes_numba(x):
    result = 0
    for i in range(x):
        result += if_prime_numba(i)
    return result

# Warm-up
sum_primes_numba(10)

if ncores == 1:
    print("\n========== Numba sequential ==========")
    suma = 0
    N = 3

    start = time.time()
    for i in range(N):
        suma = sum_primes_numba(number)
    stop = time.time()

    tiempo = (stop - start) / N

    print("The prime sum below ", number, "is ", suma,
          " and the time taken is", tiempo)
else:
    print("Skipping Numba sequential version (ncores > 1)")

In [ ]:
import multiprocessing as mp
from numba import njit

# Optimización 2: Multiprocessing con pool

@njit
def if_prime_numba_mp(x):
    if x <= 1:
        return 0
    elif x <= 3:
        return x
    elif x % 2 == 0 or x % 3 == 0:
        return 0
    i = 5
    while i * i <= x:
        if x % i == 0 or x % (i + 2) == 0:
            return 0
        i += 6
    return x

def partial_sum(args):
    start, end = args
    s = 0
    for i in range(start, end):
        s += if_prime_numba_mp(i)
    return s

def sum_primes_pool(number, nproc):
    pool = mp.Pool(processes=nproc)
    chunk = number // nproc
    ranges = [(i*chunk, (i+1)*chunk if i < nproc-1 else number) for i in range(nproc)]
    results = pool.map(partial_sum, ranges)
    pool.close()
    pool.join()
    return sum(results)

print("\n========== Multiprocessing Pool ==========")
suma = 0
N = 3

start = time.time()
for i in range(N):
    suma = sum_primes_pool(number, nproc=ncores)
stop = time.time()

tiempo = (stop - start) / N

print("The prime sum below ", number, "is ", suma, " and the time taken is", tiempo)

In [ ]:
from numba import njit, prange
import numba

# Optimización 3: Numba con prange
@njit
def if_prime_prange(x):
    if x <= 1:
        return 0
    elif x <= 3:
        return x
    elif x % 2 == 0 or x % 3 == 0:
        return 0
    i = 5
    while i * i <= x:
        if x % i == 0 or x % (i + 2) == 0:
            return 0
        i += 6
    return x

@njit(parallel=True)
def sum_primes_prange(x):
    result = 0
    for i in prange(x):
        result += if_prime_prange(i)
    return result

# Warm-up
sum_primes_prange(10)

print("\n========== Numba prange ==========")
suma = 0
N = 3

start = time.time()
for i in range(N):
    suma = sum_primes_prange(number)
stop = time.time()

tiempo = (stop - start) / N

print("The prime sum below ", number, "is ", suma, " and the time taken is", tiempo)

# Resultados y análisis:

## Resultados para number = 10⁶

**Número de núcleos: 1**

* Python secuencial
  Suma de primos: 37550402023
  Tiempo: 2.17 s

* Numba secuencial
  Suma de primos: 37550402023
  Tiempo: 0.10 s

* Multiprocessing Pool
  Suma de primos: 37550402023
  Tiempo: 0.49 s

* Numba prange
  Suma de primos: 37550402023
  Tiempo: 0.11 s

---

**Número de núcleos: 2**

* Multiprocessing Pool
  Tiempo: 0.32 s

* Numba prange
  Tiempo: 0.07 s

---

**Número de núcleos: 4**

* Multiprocessing Pool
  Tiempo: 0.23 s

* Numba prange
  Tiempo: 0.038 s

---

**Número de núcleos: 8**

* Multiprocessing Pool
  Tiempo: 0.20 s

* Numba prange
  Tiempo: 0.029 s

---

## Resultados para number = 10⁷

**Número de núcleos: 1**

* Python secuencial
  Suma de primos: 3203324994356
  Tiempo: 58.01 s

* Numba secuencial
  Suma de primos: 3203324994356
  Tiempo: 2.54 s

* Multiprocessing Pool
  Suma de primos: 3203324994356
  Tiempo: 5.21 s

* Numba prange
  Suma de primos: 3203324994356
  Tiempo: 2.87 s

---

**Número de núcleos: 2**

* Multiprocessing Pool
  Tiempo: 2.97 s

* Numba prange
  Tiempo: 1.79 s

---

**Número de núcleos: 4**

* Multiprocessing Pool
  Tiempo: 1.62 s

* Numba prange
  Tiempo: 0.96 s

---

**Número de núcleos: 8**

* Multiprocessing Pool
  Tiempo: 0.90 s

* Numba prange
  Tiempo: 0.49 s

---

# **ANÁLISIS COMPARATIVO DE RESULTADOS (10⁶ vs 10⁷)**
Los resultados muestran claramente cómo el tamaño del problema afecta la efectividad del paralelismo. Para **1 millón de números**, el código Python puro tarda 2.17 segundos, mientras que Numba secuencial lo reduce a solo 0.10 segundos, logrando una mejora de 21.5 veces con cambios mínimos. Sin embargo, al intentar paralelizar este problema pequeño, encontramos limitaciones: Multiprocessing con 8 cores alcanza 0.20 segundos (10.9x speedup) y Numba prange 0.029 segundos (75.9x speedup). Aunque numéricamente es mucha mejora, la ganancia real no es tanta porque el overhead de la paralelización consume gran parte.

La situación cambia radicalmente con **10 millones de números**. Aquí, Python puro requiere 58 segundos, evidenciando su limitación para cálculos con N mayores. Numba secuencial mantiene su buen rendimiento con 2.54 segundos (22.9x speedup), pero donde realmente brilla el paralelismo es en las versiones con varios núcelos: Multiprocasting con 8 cores logra 0.90 segundos (64.6x speedup) y Numba prange alcanza el máximo rendimiento con solo 0.49 segundos (117.9x speedup).

En definiitva, la clave para saber qué usar está en la **relación entre overhead y trabajo útil**. Para problemas pequeños como 1 millón de elementos, el coste de crear procesos o sincronizar hilos limita muchos las mejoras de tiempo, es decir, no "merece la pena". En cambio, con 10 millones de elementos, el trabajo computacional es tan grande que el overhead se vuelve insignificante en comparación, permitiendo que la paralelización muestre todo su potencial.

En resumen, Numba demuestra ser superior a Multiprocessing en ambos casos, gracias a su menor overhead (hilos vs procesos) y optimizaciones de compilación. Mientras Multiprocessing mejora de 10.9x a 64.6x speedup al aumentar el problema 10 veces, Numba prange pasa de 75.9x a 117.9x, manteniendo su ventaja.